In [1]:
import os

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.url/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：132


['https://www.dormanproducts.com/p-84706-924-351cd.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-65564-924-351.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-65557-924-352.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-129108-940-102kt.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-104645-912-605.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-105515-912-604.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-74519-924-369.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-127582-926-111cd.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-144840-940-691.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-105926-912-700.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-144839-940-690.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-74542-924-365.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-74548-924-364.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-128730-912-610.aspx?origin=ke

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
            if type_:
                with open('./part.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        continue

type_

1
2
3


'Cable - Door Latch'

In [8]:
dict_src, i, list_src = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
for src in list_src:
    if src.startswith('http'):
        dict_src[str(i)] = src
        i += 1

dict_src

{'0': 'https://static.dormanproducts.com/images/product/medium/924-351-007.JPG',
 '1': 'https://static.dormanproducts.com/images/product/medium/924-351-006.JPG',
 '2': 'https://static.dormanproducts.com/images/product/medium/924-351-005.JPG'}

In [9]:
part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

part_number

'924-351CD'

In [10]:
description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

description

'Door Latch Release Cable'

In [11]:
list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
while '' in list_text:
    list_text.remove('')

application_summary = ';'.join(list_text)

application_summary

'Ford 2024-21, 2019-92'

In [12]:
brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

brand

'Dorman - HELP! - Bagged'

In [13]:
vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
vehicle_url = f'''https://www.dormanproducts.com/{vehicle_url[0].strip()}''' if vehicle_url else ''

vehicle_url

'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84706&PartType=Cable%20-%20Door%20Latch'

In [14]:
dict_specification, list_tr = {}, html.xpath('//section[@id="productSpec"]/div/table/tr')
for i, tr in enumerate(list_tr):
    dict_specification[str(i)] = {tr.xpath('./th/text()')[0].strip().removesuffix(':').strip(): tr.xpath('./td/text()')[0].strip()}

dict_specification

{'0': {'Cable Housing Material': 'Plastic'},
 '1': {'Cable Material': 'Steel'},
 '2': {'Length': 'Long Cable - 17.05 in; Short Cable - 8.66 in'},
 '3': {'Mounting Bracket Included': 'No'},
 '4': {'Mounting Hardware Included': 'No'},
 '5': {'Package Contents': '2 Door Latch Cables'},
 '6': {'Package Quantity': '1'},
 '7': {'Packaging Type': 'Bag'},
 '8': {'Universal Or Specific Fit': 'Specific'}}

In [15]:
list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
oem = ';'.join(sorted(list(set(list_text)), key=list_text.index))

oem

'8C2Z15431A02C'

In [16]:
import json

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Type': type_,
                         'Part Number': part_number,
                         'Brand': brand,
                         'Description': description,
                         'Application Summary': application_summary,
                         'OEM': oem,
                         'Picture': '',
                         'Url': request_url,
                         'Vehicle Url': vehicle_url,
                         'Json_Src': json.dumps(dict_src),
                         'Json_Specification': json.dumps(dict_specification)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,Brand,Part Number,Description,Application Summary,OEM,Picture,Url,Vehicle Url,Json_Src,Json_Specification
0,1,Cable - Door Latch,Dorman - HELP! - Bagged,924-351CD,Door Latch Release Cable,"Ford 2024-21, 2019-92",8C2Z15431A02C,,https://www.dormanproducts.com/p-84706-924-351...,https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": ""https://static.dormanproducts.com/image...","{""0"": {""Cable Housing Material"": ""Plastic""}, ""..."


In [17]:
crawler_status = 'ok'

crawler_status

'ok'

In [18]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [19]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Type,Brand,Part Number,Description,Application Summary,OEM,Picture,Url,Vehicle Url,Json_Src,Json_Specification
0,1,Cable - Door Latch,Dorman - HELP! - Bagged,924-351CD,Door Latch Release Cable,"Ford 2024-21, 2019-92",8C2Z15431A02C,,https://www.dormanproducts.com/p-84706-924-351...,https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": ""https://static.dormanproducts.com/image...","{""0"": {""Cable Housing Material"": ""Plastic""}, ""..."


In [20]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
